## Градиент стратегии: REINFORCE.

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Если использовать метод Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$ отдачу $R_t$, то тогда происходит переход к алгоритму REINFORCE и обновление весов будет осуществляться по правилу:

$$\nabla_\theta J(\theta) = [R_t \nabla_\theta \ln \pi_\theta(A_t \vert S_t)]$$

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Основной цикл

In [ ]:
def print_mean_reward(step, episode_rewards):
    if not episode_rewards:
        return

    t = min(50, len(episode_rewards))
    mean_reward = sum(episode_rewards[-t:]) / t
    print(f"step: {str(step).zfill(6)}, mean reward: {mean_reward:.2f}")
    return mean_reward


class Rollout:
    def __init__(self):
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def append(self, log_prob, reward, done):
        self.logprobs.append(log_prob)
        self.rewards.append(reward)
        self.is_terminals.append(done)


def run(env: gym.Env, hidden_size: int, lr: float, gamma: float, max_episodes: int, rollout_size: int):
    # Инициализируйте агента `agent`
    """<codehere>"""
    agent = ReinforceAgent(env.observation_space.shape[0], env.action_space.n, hidden_size, lr, gamma)
    """</codehere>"""

    step = 0
    rollout = Rollout()
    episode_rewards = []

    for i_episode in range(1, max_episodes + 1):
        cumulative_reward = 0
        terminated = False
        state, _ = env.reset()

        while not terminated:
            step += 1

            action, log_prob = agent.act(state)
            state, reward, terminated, truncated, _ = env.step(action)

            # сохраняем награды и флаги терминальных состояний:
            rollout.append(log_prob, reward, terminated)
            cumulative_reward += reward
            terminated |= truncated

        episode_rewards.append(cumulative_reward)

        # выполняем обновление
        if len(rollout.rewards) >= rollout_size:
            agent.update(rollout)
            mean_reward = print_mean_reward(step, episode_rewards)
            if mean_reward >= 200:
                print('Принято!')
                return
            rollout = Rollout()
            episode_rewards = []

In [ ]:
# Реализуйте класс, задающий стратегию агента.
# Подсказки:
#     1) можно воспользоваться базовым классом `torch.nn.Module`,
#     2) размер нейронной сети можно выбрать таким: (input_dim, hidden_dim, output_dim),
#     3) в качестве функции активации возьмите гиперболический тангенс или ReLU
#     4) подумайте, как получить на выходе из нейронной сети вероятности действий,
#     5) для выбора действия в соответствии со стратегией, можно воспользоваться `torch.distributions.Categorical`
#     6) помните, что помимо самого действия вам позже также пригодится логарифм его вероятности
"""<codehere>"""
class MLPModel(nn.Module):
    def __init__(self, state_dim, action_dim, hidden):
        super().__init__()
        # Определяем сеть для предсказания действий,
        # на вход вектор по размеру состояний
        # на выходе вектор по размеру действий
        self.net =

    def forward(self, state):
        # Предсказываем действия
        state =
        action_probs =

        dist = Categorical(action_probs)
        action = dist.sample()

        return action.item(), dist.log_prob(action)
"""</codehere>"""


class ReinforceAgent:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, gamma):
        self.lr = lr
        self.gamma = gamma

        # Инициализируйте стратегию агента и SGD оптимизатор (например, `torch.optim.Adam`)
        """<codehere>"""

        """</codehere>"""

    def act(self, state):
        # Произведите выбор действия и верните кортеж (действие, логарифм вероятности этого действия)
        """<codehere>"""

        """</codehere>"""

    def update(self, rollout: Rollout):
        # Конвертируйте накопленный список вознаграждений в список отдач. Назовем его `rewards`
        # Подсказки:
        #    1) обход списка стоит делать в обратном порядке,
        #    2) не забывайте сбрасывать отдачу при окончании эпизода
        # rewards =
        """<codehere>"""



        """</codehere>"""

        # Выполните нормализацию вознаграждений (отдач) через отклонение
        # rewards =
        """<codehere>"""

        """</codehere>"""

        # Вычислите ошибку `loss` и произведите шаг обновления градиентным спуском
        # Подсказки: используйте `.to(device)`, чтобы разместить тензор на соотв. цпу/гпу
        """<codehere>"""

        print(f'L: {loss}')

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        """</codehere>"""

### Определяем гиперпараметры и запускаем обучение

In [ ]:
from gymnasium.wrappers.time_limit import TimeLimit
env_name = "CartPole-v1"

run(
    env = TimeLimit(gym.make(env_name), 1000),
    max_episodes = 50000,  # количество эпизодов обучения
    hidden_size = 64,  # кол-во переменных в скрытых слоях
    rollout_size = 500,  # через столько шагов стратегия будет обновляться
    lr = 0.01, # learning rate
    gamma = 0.995,  # дисконтирующий множитель,
)

L: 0.006624714937061071
step: 000501, mean reward: 19.27
L: -0.009398132562637329
step: 001002, mean reward: 21.78
L: -0.014965783804655075
step: 001528, mean reward: 32.88
L: -0.0019939704798161983
step: 002082, mean reward: 55.40
L: 0.004818425979465246
step: 002623, mean reward: 54.10
L: -0.0022677441593259573
step: 003144, mean reward: 43.42
L: 0.003046266036108136
step: 003693, mean reward: 54.90
L: -0.003125885734334588
step: 004221, mean reward: 58.67
L: -0.0093100406229496
step: 004723, mean reward: 62.75
L: -0.0051866876892745495
step: 005306, mean reward: 83.29
L: 0.018178287893533707
step: 005806, mean reward: 62.50
L: 0.0035867877304553986
step: 006369, mean reward: 70.38
L: 0.0006288680015131831
step: 006904, mean reward: 89.17
L: 0.006145173218101263
step: 007435, mean reward: 106.20
L: -0.04876646026968956
step: 007988, mean reward: 110.60
L: -0.0015780149260535836
step: 008596, mean reward: 152.00
L: -0.019675662741065025
step: 009162, mean reward: 141.50
L: -0.02831821

In [ ]:
"""<comment>"""
from codehere import convert
import os

os.makedirs('render', exist_ok=True)

nb_name = 'reinforce'
convert(
    file=f'{nb_name}.ipynb',
    outfile=f'render/{nb_name}.ipynb',
    clear=True, replacement=" Здесь ваш код "
)
convert(
    file=f'{nb_name}.ipynb',
    outfile=f'render/{nb_name}-solution.ipynb',
    clear=True, solution=True, replacement=" Здесь ваш код "
)
"""</comment>"""

Saved in:  render/reinforce.ipynb
Saved in:  render/reinforce-solution.ipynb


'</comment>'